# Concurrent computing.

## Isaac Fernandez

The following implementation is a solution to the so called "single lane problem". 

In [3]:
import threading
import time
import multiprocessing as mp
import sys

class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class Bridge():
    def __init__(self):    
        self.lock = threading.Condition()
        self.taken = False

    def take(self, direction):         # used for synchronization
        with self.lock:
            while self.taken == True:
                sys.stdout.write("-------------Car [%s] waiting----------\n" % (direction))
                self.lock.wait()
            self.taken = True
            sys.stdout.write("--------Car[%s] is crossing the bridge-----\n"%(direction))
            sys.stdout.write("-------------Car[%s] crossed the bridge----------\n"%(direction))
            self.lock.notifyAll()

    def drop(self, sentido):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.taken = False
            self.lock.notifyAll()
            
class left_lane(threading.Thread):
    def __init__(self, left, semaphore):
        threading.Thread.__init__(self)
        self.left = left
        self.semaphore = semaphore
    
    def run(self):
        while True:
            sys.stdout.write("Car on the left lane waiting\n" )
            time.sleep(1)
            self.semaphore.down()              
            time.sleep(1)
            self.left.take('left')        
            time.sleep(1)  #           sys.stdout.write("-------------Carro[izquierdo] pasó por el puente----------\n")
            self.left.drop('left')  
            time.sleep(1)
            self.semaphore.up()             
            time.sleep(1)
            
class right_lane(threading.Thread):
    def __init__(self, right, semaphore):
        threading.Thread.__init__(self)
        self.right = right
        self.semaphore = semaphore
    
    def run(self):
        for i in range(1):
            sys.stdout.write("Car on the right lane waiting\n")
            time.sleep(1)
            self.semaphore.down()              
            time.sleep(1)
            self.right.take('right')        
            time.sleep(1)  #sys.stdout.write("-------------Carro[derecho] pasó por el puente----------\n")
            self.right.drop('right')  
            time.sleep(1)
            self.semaphore.up()             
            time.sleep(1)

def car_bridge():
    n_left = 5
    n_right = 5
    c = [Bridge() for i in range(n_left + n_right)]
    
    semaphore = Semaphore(2)
    sys.stdout.write("\n")
    p_left = [left_lane(c[i], semaphore) for i in range(n_left)]
    p_right = [right_lane(c[i], semaphore) for i in range(n_right)]

    for i in range(n_left):
        p_left[i].start()
    for i in range(n_right):
        p_right[i].start()
        
    for i in range(n_left):
        p_right[i].join()
    for i in range(n_right):
        p_left[i].join()
    
car_bridge()


Car on the left lane waiting
Car on the left lane waiting
Car on the left lane waiting
Car on the left lane waiting
Car on the left lane waiting
Car on the rigth lane waiting
Car on the rigth lane waiting
Car on the rigth lane waiting
Car on the rigth lane waiting
Car on the rigth lane waiting
--------Car[right] is crossing the bridge-----
--------Car[left] is crossing the bridge-----
-------------Car[left] crossed the bridge----------
-------------Car[right] crossed the bridge----------
Car on the left lane waiting
--------Car[right] is crossing the bridge-----
-------------Car[right] crossed the bridge----------
--------Car[left] is crossing the bridge-----
-------------Car[left] crossed the bridge----------
Car on the left lane waiting
--------Car[left] is crossing the bridge-----
-------------Car[left] crossed the bridge----------
--------Car[right] is crossing the bridge-----
-------------Car[right] crossed the bridge----------
--------Car[left] is crossing the bridge-----
Car on

KeyboardInterrupt: 